[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/grounded-segment-anything-colab/blob/main/grounded-segment-anything-colab.ipynb)

In [ ]:
%cd /content
!pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.18 triton==2.0.0 -U

!apt-get -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/ovseg/resolve/main/sam_vit_h_4b8939.pth -d /content -o sam_vit_h_4b8939.pth

!git clone -b dev https://github.com/camenduru/Grounded-Segment-Anything
%cd /content/Grounded-Segment-Anything
!pip install -r requirements.txt
%cd /content/Grounded-Segment-Anything/GroundingDINO
!pip install .
%cd /content/Grounded-Segment-Anything/segment_anything
!pip install .

import os
import shlex
import subprocess
from pathlib import Path
from typing import Union

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=True)
    path.chmod(0o600)

ssh_name = "id_rsa"
ssh_path = Path(os.path.dirname(os.getcwd())) / ssh_name
gen_key(ssh_path)

!ssh-keygen -t rsa -b 4096 -N "" -q -f /content/id_rsa -y

In [ ]:
%cd /content/Grounded-Segment-Anything

import threading
def tunnel():
    !ssh -R 80:127.0.0.1:7860 -o StrictHostKeyChecking=no -i /content/id_rsa remote.moe
threading.Thread(target=tunnel, daemon=True).start()

!python gradio_app.py --debug --share